In [1]:
import sys

# setting path
sys.path.append('../')

import livef1
import pandas as pd

In [2]:
season = livef1.get_season(2021)

In [3]:
print(season.meetings_table.dtypes)
season.meetings_table.head()

season_year                           int64
meeting_code                         object
meeting_number                        int64
meeting_location                     object
meeting_offname                      object
meeting_name                         object
meeting_country_key                   int64
meeting_country_code                 object
meeting_country_name                 object
meeting_circuit_key                   int64
meeting_circuit_shortname            object
session_key                           int64
session_type                         object
session_name                         object
session_startDate            datetime64[ns]
session_endDate              datetime64[ns]
gmtoffset                            object
path                                 object
dtype: object


,season_year,meeting_code,meeting_number,meeting_location,meeting_offname,meeting_name,meeting_country_key,meeting_country_code,meeting_country_name,meeting_circuit_key,meeting_circuit_shortname,session_key,session_type,session_name,session_startDate,session_endDate,gmtoffset,path
meeting_key,,,,,,,,,,,,,,,,,,
1064,2021,BRN0104,1,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,Bahrain Grand Prix,36,BRN,Bahrain,63,Sakhir,6210,Practice 1,Practice 1,2021-03-26 14:30:00,2021-03-26 15:30:00,03:00:00,2021/2021-03-28_Bahrain_Grand_Prix/2021-03-26_...
1064,2021,BRN0104,1,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,Bahrain Grand Prix,36,BRN,Bahrain,63,Sakhir,6211,Practice 2,Practice 2,2021-03-26 18:00:00,2021-03-26 19:00:00,03:00:00,2021/2021-03-28_Bahrain_Grand_Prix/2021-03-26_...
1064,2021,BRN0104,1,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,Bahrain Grand Prix,36,BRN,Bahrain,63,Sakhir,6212,Practice 3,Practice 3,2021-03-27 15:00:00,2021-03-27 16:00:00,03:00:00,2021/2021-03-28_Bahrain_Grand_Prix/2021-03-27_...
1064,2021,BRN0104,1,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,Bahrain Grand Prix,36,BRN,Bahrain,63,Sakhir,6213,Qualifying,Qualifying,2021-03-27 18:00:00,2021-03-27 19:00:00,03:00:00,2021/2021-03-28_Bahrain_Grand_Prix/2021-03-27_...
1064,2021,BRN0104,1,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2021,Bahrain Grand Prix,36,BRN,Bahrain,63,Sakhir,6214,Race,Race,2021-03-28 18:00:00,2021-03-28 20:00:00,03:00:00,2021/2021-03-28_Bahrain_Grand_Prix/2021-03-28_...


In [4]:
print(season.season_table.to_markdown())

|    |   Meeting Key | Meeting Code   | Meeting Name              | Meeting Circuit Shortname     |   No. Sessions | Race Startdate      |
|---:|--------------:|:---------------|:--------------------------|:------------------------------|---------------:|:--------------------|
|  0 |          1064 | BRN0104        | Bahrain Grand Prix        | Sakhir                        |              5 | 2021-03-28 18:00:00 |
|  1 |          1065 | ITA0110        | Emilia Romagna Grand Prix | Imola                         |              5 | 2021-04-18 15:00:00 |
|  2 |          1066 | POR0401        | Portuguese Grand Prix     | Algarve International Circuit |              5 | 2021-05-02 15:00:00 |
|  3 |          1086 | ESP0111        | Spanish Grand Prix        | Catalunya                     |              5 | 2021-05-09 15:00:00 |
|  4 |          1067 | MON0112        | Monaco Grand Prix         | Monte Carlo                   |              5 | 2021-05-23 15:00:00 |
|  5 |          1068 | AZE0

In [5]:
meeting = livef1.get_meeting(season=2024, meeting_identifier="Monza")

2024-12-03 16:46:05,513 - livef1.utils.logger - INFO - ..:: Search started.
2024-12-03 16:46:05,524 - livef1.utils.logger - INFO - ..:: Found.
2024-12-03 16:46:05,526 - livef1.utils.logger - INFO - ..:: Most similar: Monza
2024-12-03 16:46:05,527 - livef1.utils.logger - INFO - ..:: Row: 30
2024-12-03 16:46:05,529 - livef1.utils.logger - INFO - ..:: Column: meeting_circuit_shortname


In [6]:
season.meetings_table[season.meetings_table["session_type"] == "Race"]["session_startDate"]

meeting_key
1064   2021-03-28 18:00:00
1065   2021-04-18 15:00:00
1066   2021-05-02 15:00:00
1086   2021-05-09 15:00:00
1067   2021-05-23 15:00:00
1068   2021-06-06 16:00:00
1070   2021-06-20 15:00:00
1092   2021-06-27 15:00:00
1071   2021-07-04 15:00:00
1072   2021-07-18 15:00:00
1073   2021-08-01 15:00:00
1074   2021-08-29 15:00:00
1075   2021-09-05 15:00:00
1076   2021-09-12 15:00:00
1077   2021-09-26 15:00:00
1078   2021-10-10 15:00:00
1102   2021-10-24 14:00:00
1103   2021-11-07 13:00:00
1104   2021-11-14 14:00:00
1105   2021-11-21 17:00:00
1106   2021-12-05 20:30:00
1106   2022-03-24 19:00:00
1107   2021-12-12 17:00:00
Name: session_startDate, dtype: datetime64[ns]

In [7]:


season.meetings_table[season.meetings_table["meeting_code"] == "AUT0108"]

,season_year,meeting_code,meeting_number,meeting_location,meeting_offname,meeting_name,meeting_country_key,meeting_country_code,meeting_country_name,meeting_circuit_key,meeting_circuit_shortname,session_key,session_type,session_name,session_startDate,session_endDate,gmtoffset,path
meeting_key,,,,,,,,,,,,,,,,,,
1092,2021,AUT0108,8,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,Styrian Grand Prix,17,AUT,Austria,19,Spielberg,6418,Practice 1,Practice 1,2021-06-25 11:30:00,2021-06-25 12:30:00,02:00:00,2021/2021-06-27_Styrian_Grand_Prix/2021-06-25_...
1092,2021,AUT0108,8,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,Styrian Grand Prix,17,AUT,Austria,19,Spielberg,6419,Practice 2,Practice 2,2021-06-25 15:00:00,2021-06-25 16:00:00,02:00:00,2021/2021-06-27_Styrian_Grand_Prix/2021-06-25_...
1092,2021,AUT0108,8,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,Styrian Grand Prix,17,AUT,Austria,19,Spielberg,6420,Practice 3,Practice 3,2021-06-26 12:00:00,2021-06-26 13:00:00,02:00:00,2021/2021-06-27_Styrian_Grand_Prix/2021-06-26_...
1092,2021,AUT0108,8,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,Styrian Grand Prix,17,AUT,Austria,19,Spielberg,6421,Qualifying,Qualifying,2021-06-26 15:00:00,2021-06-26 16:00:00,02:00:00,2021/2021-06-27_Styrian_Grand_Prix/2021-06-26_...
1092,2021,AUT0108,8,Spielberg,FORMULA 1 BWT GROSSER PREIS DER STEIERMARK 2021,Styrian Grand Prix,17,AUT,Austria,19,Spielberg,6417,Race,Race,2021-06-27 15:00:00,2021-06-27 17:00:00,02:00:00,2021/2021-06-27_Styrian_Grand_Prix/2021-06-27_...
1071,2021,AUT0108,9,Spielberg,FORMULA 1 BWT GROSSER PREIS VON ÖSTERREICH 2021,Austrian Grand Prix,17,AUT,Austria,19,Spielberg,6245,Practice 1,Practice 1,2021-07-02 11:30:00,2021-07-02 12:30:00,02:00:00,2021/2021-07-04_Austrian_Grand_Prix/2021-07-02...
1071,2021,AUT0108,9,Spielberg,FORMULA 1 BWT GROSSER PREIS VON ÖSTERREICH 2021,Austrian Grand Prix,17,AUT,Austria,19,Spielberg,6246,Practice 2,Practice 2,2021-07-02 15:00:00,2021-07-02 16:00:00,02:00:00,2021/2021-07-04_Austrian_Grand_Prix/2021-07-02...
1071,2021,AUT0108,9,Spielberg,FORMULA 1 BWT GROSSER PREIS VON ÖSTERREICH 2021,Austrian Grand Prix,17,AUT,Austria,19,Spielberg,6247,Practice 3,Practice 3,2021-07-03 12:00:00,2021-07-03 13:00:00,02:00:00,2021/2021-07-04_Austrian_Grand_Prix/2021-07-03...
1071,2021,AUT0108,9,Spielberg,FORMULA 1 BWT GROSSER PREIS VON ÖSTERREICH 2021,Austrian Grand Prix,17,AUT,Austria,19,Spielberg,6248,Qualifying,Qualifying,2021-07-03 15:00:00,2021-07-03 16:00:00,02:00:00,2021/2021-07-04_Austrian_Grand_Prix/2021-07-03...
